In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import fnmatch
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ruptures as rpt


In [ ]:
from lib.data_classes.pffpFolder import pffpDataFolder
from lib.signal_processing.signal_function import moving_average, find_deriv_change, find_drops
from lib.general_functions.global_constants import GRAVITY_CONST
from lib.general_functions.helper_function import convert_time_units

In [ ]:
# Testing the binary file classa
#TODO: for some reason I need the full path to read the files, I'll need to look into this in the future
# folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\test"
# folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\Mouth 1" #pffp 3, sequim 2023
# folder_dir = r"/home/sandbiter/Documents/BlueDrop_Analysis_Lib/Potomac_to Jonathan"
# folder_dir = r"/home/sandbiter/Documents/BlueDrop_Analysis_Lib/test_folder"
# folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\Clay_Air_Drops_GBE_AdamsPoint"
# folder_dir = r"/home/sandbiter/Documents/BlueDrop_Analysis_Lib/Mouth_1"
# folder_dir = r"/home/sandbiter/Documents/BlueDrop_Analysis_Lib/Clay_Air_Drops_GBE_AdamsPoint"
folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\stark_drops"
# folder_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\Clay_air_drops\Clay_Air" #1, march 2023
calibration_dir = r"calibration_factors\BlueDrop_Calibration_Factors.xlsx"

In [ ]:
tip_dir = r"C:\Geotech_Research\Field_Analysis\BlueDrop_Analysis_Lib\calibration_factors\BluedDrop_Cone_Values.xlsx"


In [ ]:
# TODO: Combine the pffp_id and the read_calibration_excel_sheet just have the objet call inside
name = "test"
id = 9
test_folder = pffpDataFolder(folder_dir, pffp_id = id, calibration_factor_dir=calibration_dir, survey_name = name)
print(test_folder)

In [ ]:
# Read the calibration data
test_folder.read_calibration_excel_sheet()
test_folder.calibration_excel_sheet

# Select the specific calibration params
if id == 3:
    test_folder.get_sensor_calibration_params("Sequim_2021")
elif id == 1:
    test_folder.get_sensor_calibration_params("March_2023")
elif id ==9:
    test_folder.get_sensor_calibration_params("October_2023")
else:
    raise ValueError("not a valid pffp id")

In [ ]:
# Init a pffpFile instances for each binary file and store the results in the folder
test_folder.store_pffp_files(recursive = False)

print(test_folder) # Information about the folder

In [ ]:
# Analyze the selected pffp files
test_folder.analyze_all_files(store_df=True, use_pore_pressure=True, 
                              select_accel=["2g_accel", "50g_accel", "250g_accel"],
                              debug = False)


In [ ]:
print(test_folder)

In [ ]:
test_folder.process_drop_files()

In [ ]:
file = test_folder.pffp_drop_files[0]
# file.manually_process_drops()
print(file)

#### Start Bearing Capcity Analysis

In [ ]:
drop = file.drops[0]
pffp_id = test_folder.pffp_id
drop.get_pffp_tip_values(pffp_id=pffp_id, tip_type="cone", date_string = "April_2024", file_dir = tip_dir)

In [ ]:
print(drop)

In [ ]:
file.quick_view(interactive=True)

In [ ]:
drop.quick_view_impulse()

In [ ]:
drop.quick_view_release()

In [ ]:
drop.water_drop = True
drop.convert_tip_vals()

# for name in ["projected", "mantle"]:
#     drop.calc_drop_contact_area(area_type = name)
#     drop.calc_drop_dynamic_bearing()
#     drop.calc_drop_qs_bearing(k_factor = 0.3)

In [ ]:
drop.output_impulse_data(folder_dir = "stark_drops\processed_data")
drop.output_release_data(folder_dir = "stark_drops\processed_data")
# drop.output_bearing_data(df= self.folder_dir = "stark_drops\processed_data")

In [ ]:
from lib.machine_learning.randomForestClass import RandForest

ML_model_dir = r"lib\machine_learning\machine_learning_models\rand_forest_prior_model.joblib"
rand_forest_model = RandForest(name="{} drop index:{}".format(file.file_name, drop.file_drop_index), model_dir=ML_model_dir ) 

In [ ]:
drop.units

In [ ]:
from lib.general_functions.helper_function import convert_accel_units, convert_length_units

In [ ]:
accel_unit = drop.units["accel"]
displacement_unit = drop.units["displacement"]
max_displacement = drop.impulse_df["displacement"].max()
max_accel = drop.impulse_df["accel"].max()

max_accel = convert_accel_units(max_accel, accel_unit, "g")  # Convert to g's
max_displacement = convert_length_units(max_displacement, displacement_unit, "cm") # Convert to centimeters

output= rand_forest_model.predict_probability(accel_data=max_accel, max_displacement=max_displacement)[0]


In [ ]:
rand_forest_model.plot_prediction(output, fig_size=[4,3], 
                                  title = "{} drop index:{}".format("file", drop.file_drop_index), save = True, name = "sediment_prediction")

### Things that need to be done
 
* Trim the top of the bearing data plot (doesn't make a lot of sense but sure)
  
* Create the a test database that has:
  * base_data
  * accel_data
  
* Add on the following tables
  * bearing_capacity
  
* Add on the followinig tables
  * bluedrop config
  * bluedrop calibration data



### Options for formattig data
* Use a JSON for a quickoverview of the survey metadata
* Use a csv format to store information about each of the drops